In [1]:
import dask.dataframe as dask_pd
import dask_geopandas as dask_gpd
import geopandas as gpd
from dask.distributed import Client

In [2]:
explore = dask_pd.read_csv("sample-folder/12-8008/ue_traffic.zip", usecols=[0,1,2,3,4,10,12,13], assume_missing=True)
# explore = dask_pd.read_csv("sample-folder/mdt_result.csv", assume_missing=True)
boundary = dask_pd.read_csv("csv_polygon/polygon_xl-eid_LautIncluded.csv", assume_missing=True)
grid = dask_pd.read_csv('Grid_Folder/40x40grid_alljabo.csv', assume_missing=True)
# grid = dask_pd.read_parquet('Grid_Folder/100x100_gridjabo.parquet', assume_missing=True)

/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/dask/dataframe/io/csv.py:544: UserWarning: Warning zip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


In [3]:
explore

,date,hour,site,enodebid,ci,ue_throughput_dl_drb_kbps,longitude,latitude
npartitions=1,,,,,,,,
,float64,object,object,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


In [3]:
explore = explore.reset_index()

In [4]:
explore['pointer'] = "POINT (" + explore['longitude'].astype(str) + " " + explore['latitude'].astype(str) + ")"

dask_gdf_mdt = dask_gpd.from_dask_dataframe(explore, geometry=explore["pointer"].map_partitions(gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)
#dask_gdf_mdt = dask_gdf_mdt.set_crs(4326)

In [5]:
dask_gdf_boundary = dask_gpd.from_dask_dataframe(boundary, geometry=boundary['WKT'].map_partitions(gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)

In [6]:
dask_gdf_grid = dask_gpd.from_dask_dataframe(grid, geometry=grid['geometry'].map_partitions(gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)

In [7]:
dask_gdf_grid = dask_gdf_grid.reset_index()
dask_gdf_grid = dask_gdf_grid.set_index(dask_gdf_grid['index'])

In [ ]:
dask_gdf_mdt = dask_gdf_mdt.reset_index()
dask_gdf_mdt = dask_gdf_mdt.set_index(dask_gdf_mdt['index'])

In [ ]:
boolvalue = dask_gdf_mdt.within(dask_gdf_grid)

# dask_gdf_mdt = dask_gdf_mdt.sjoin(dask_gdf_boundary, how='inner', predicate='within')
# dask_gdf_mdt = dask_gdf_mdt.sjoin(dask_gdf_boundary, how='inner', predicate='within')

In [ ]:
dask_gdf_mdt.multi

In [ ]:
dask_gdf_mdt = dask_gdf_mdt.drop(columns=['index_right', 'id'])

In [ ]:
dask_gdf_mdt = dask_gdf_mdt.sjoin(dask_gdf_grid, how='inner', predicate='within')